In [1]:
import pandas as pd

# custom
import get_govorg_list
import govorg_matcher

# spacy
import spacy
from spacy_lookup import Entity

# Import the English language class
from spacy.lang.en import English

# make use of widescreen
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# trial with a manageable number of GOV.UK pages
content_n = 1000

In [2]:
# our list of all Government Organisations from the Register
ORG_GOV = get_govorg_list.main()

In [3]:
nlp = spacy.load('en_core_web_sm')
entity = Entity(keywords_list=ORG_GOV)
nlp.add_pipe(entity, last=True)

# available from GOV.UK data scientists
# data has been pre-processed for taxonomy work
# this loses us useful information such as capital letters
# will want to adjust pre-processing pipeline
df = pd.read_csv("data/11-02-19/labelled.csv",
                 usecols=["base_path", "content_id",
                          "description", "locale",
                          "title", "body", "combined_text"],
                nrows = content_n)

In [5]:
def text_gov_org_match(text):
    """Return a list of GOV_ORG entities found at least once in a str."""
    doc = nlp(text)
    # remove any duplicates because dictionaries cannot have duplicate keys
    list_of_gov_org_entities_with_duplicates = [token._.canonical for token in doc if token._.is_entity]
    list_no_duplicates = list(dict.fromkeys(list_of_gov_org_entities_with_duplicates))
    return list_no_duplicates

In [6]:
# need to fix function so it can be called from script
text_gov_org_match(df.at[3,"body"])

['The Charity Commission']